### GENRE Output Reformat

This notebook reformats GENRE's output and saves it to a csv which is more easily interpretable by evaluation scripts

In [1]:
import pandas as pd
import re
import pickle

In [2]:
result_df = pd.read_csv('FAA_model_gold_standards_independant_20240611150355.csv')
result_df.head()

,c5_unique_id,c119_text,c119_output,c119_score
0,19990213001379A,ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CON...,ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST { C...,-0.553834
1,19800217031649I,"AFTER TAKEOFF, ENGINE QUIT. WING FUEL TANK SUM...","AFTER TAKEOFF, ENGINE QUIT. WING FUEL TANK SUM...",-0.262258
2,19790720021329A,HELICOPTER TOOK OFF WITH SLING LOAD ATTACHED. ...,HELICOPTER TOOK OFF WITH SLING LOAD ATTACHED. ...,-0.145806
3,19841214074599I,WHILE TAXIING LOST NOSEWHEEL STEERING AND BRAK...,WHILE TAXIING LOST NOSEWHEEL STEERING AND BRAK...,-0.068722
4,19860128014289I,FORWARD CARGO DOOR OPENED AS AIRCRAFT TOOK OFF...,FORWARD CARGO DOOR OPENED AS AIRCRAFT TOOK OFF...,-0.152180


**Load title2wikidataID dict**

We follow the example at https://github.com/facebookresearch/GENRE/tree/main/examples_mgenre and download lang_title2wikidataID-normalized_with_redirect.pkl to translate the generated titles to Wikidata QIDs

In [ ]:
with open("lang_title2wikidataID-normalized_with_redirect.pkl", "rb") as f:
    lang_title2wikidataID = pickle.load(f)

In [ ]:
list(lang_title2wikidataID.keys())[10:20] # see sample

In [ ]:
('en','Boeing') in lang_title2wikidataID

In [ ]:
max(lang_title2wikidataID[('en','Boeing')], key=lambda y: int(y[1:])) # func from mgenre examples notebook

**Define title2id function**

Flexible for capitalization issues

In [ ]:
def title2id(title):
    
    try:
        id = max(lang_title2wikidataID[('en',title)], key=lambda y: int(y[1:]))
    except KeyError:
        try:
            id = max(lang_title2wikidataID[('en',title.title())], key=lambda y: int(y[1:]))
        except:
            id = None
    return id

**Get entities, titles, and id using regex**

In [ ]:
out_dict = {'c5_id':[],'c119_text':[],'c119_output':[],'c119_score':[], 'entities':[],'titles':[],'qids':[]}
ent_title_p = re.compile('[^\{]*\{ ([^\}]+) \} \[ ([^\]]+) ] (.*)') # collects groups: ent, title, rest

for i in range(len(result_df)):
    text = result_df['c119_output'].iat[i]
    
    while text:
        mo = re.match(ent_title_p, text)
        if mo:
            ent, title, text = mo.groups()

            id = title2id(title)
    
            out_dict['c5_id'].append(result_df['c5_unique_id'].iat[i])
            out_dict['c119_text'].append(result_df['c119_text'].iat[i])
            out_dict['c119_output'].append(result_df['c119_output'].iat[i])
            out_dict['c119_score'].append(result_df['c119_score'].iat[i])
            out_dict['mentions'].append(ent)
            out_dict['titles'].append(title)
            out_dict['qids'].append(id)
        
        else:
            text = None

In [ ]:
out_df = pd.DataFrame(out_dict)
out_df.head()

**Save DataFrame**

In [ ]:
out_df.to_csv('../../tool_results/genre/genre.csv', index=False)